In [1]:
import numpy as np
import random
# import trimesh
from mpl_toolkits import mplot3d
from matplotlib import pyplot
import copy
import os
import matplotlib.patches as mpatches
import sys
import time
import statistics



In [2]:

def initial_cell(state,Size):
    if Size == 2:
        state['CellType'].append('SC')
        state['Volume'].append(1.82)

    elif Size == 1:
        state['CellType'].append('bc')
        state['Volume'].append(0.5571)
    return state

def cell(state,Size):
    if Size == 2:
        rand_cell = np.random.randint(0,3)#Random cell type
        if rand_cell == 0: #body centered
            state['CellType'].append('TR')
            state['Volume'].append(3.02)
        elif rand_cell == 1: #body centered
            state['CellType'].append('SC')
            state['Volume'].append(1.82)
        elif rand_cell == 2:  #simple cubic with rounded edges
            state['CellType'].append('FC')
            state['Volume'].append(3.69)
    elif Size == 1:
        rand_cell = np.random.randint(0,3)#Random cell type
        if rand_cell == 0:  #simple cubic with rounded edges
            state['CellType'].append('sol')
            state['Volume'].append(1)
        elif rand_cell == 1:  #simple cubic with rounded edges
            state['CellType'].append('sc')
            state['Volume'].append(0.7839)
        elif rand_cell == 2:  #simple cubic with rounded edges
            state['CellType'].append('bc')
            state['Volume'].append(0.5571)
    return state

In [3]:
def constraint(State,V_max):#,A_max):
    VOL = np.array(State['Volume']).reshape(-1,1)
    CELL =  np.array(State['CellType']).reshape(-1,1)
    POS = np.array(State['Position'])
    cell = CELL.flatten()
    V_sup = sum(VOL)
    if V_sup > V_max:
        ver = False
#         print('Volume Violation:',V_sup,'>',V_max)
    else:
        ver = True
    return V_sup,ver

In [4]:
def initialize(Unit,Sup,init):
    T_state = {'CellType':[],'Volume':[],'Position':[],'Size':[]}
    for i in range(len(Unit)):
        new_cell = {'CellType':[],'Volume':[]}
        if i<len(Sup):
            size = 2
        else:
            size = 1
        new_cell = initial_cell(new_cell,size)

#         new_cell = cell(new_cell,size)
        T_state['CellType'].append(new_cell['CellType'][0])
        T_state['Volume'].append(new_cell['Volume'][0])
        T_state['Position'].append(Unit[i].tolist())
        T_state['Size'].append(size)
    Vol,ver = constraint(T_state,0)#,0)
    return Vol,T_state

def evaluate(State,init,DisMax,StrMax):   #Objectives
    num_cells=len(State['CellType'])
    CELL = np.array(State['CellType']).reshape(-1,1)
    POS = np.array(State['Position'])*1e-3
    Size = np.array(State['Size'])
    h =1e-3
    Dis, Stress, E = FEM_Thermal(CELL,POS,Size,h,init,DisMax,StrMax)   
#     Dis = 1
#     Stress = 1
#     E = 1
    return E,init,Dis,Stress

In [5]:
def metopolis(E_prev,Vol,Vol2, new_state,prev_state,T,init,Dis,Str,DisMax,StrMax): #boolean of True/False
    E1 = E_prev
    Vol1=Vol
    Dis1 = Dis
    Str1 = Str
    E2,init,Dis2,Str2= evaluate(new_state,init,DisMax,StrMax)
    if E1 < E2:
        P_acc = np.exp(-(E2-E1)/T) #minimize
        comp = np.random.uniform(0, 1)
        if P_acc > comp:
            accept = True
            E = E2
            Vol = Vol2
            Dis = Dis2
            Str = Str2
        else:
            accept = False
            P_acc = 0
            E = E1
            Vol = Vol1
            Dis = Dis1
            Str = Str1
    elif E2 == False:
        accept = False
        P_acc = 0
        E = E1
        Vol = Vol1
        Dis = Dis1
        Str = Str1
    else:
        P_acc = 1
        accept = True
        E = E2
        Vol = Vol2
        Dis = Dis2
        Str = Str2
    return accept,E, Vol, P_acc,Dis, Str

In [6]:
def ShapeAnneal(it,state,T,limit,n,alpha,Sup,Unit,E,Vol,init,V_max,Dis,Str,DisMax,StrMax):
    T_0 = T
    P_acc = 1
    Res_loss = []
    E_loss = []
    V_loss = []
#     TotalArea_loss = []
    Str_loss = []
    Dis_loss = []
    V_iter = []
    Temp = []
    Samples = []
    P_acc_iter = []
    Str_iter = []
    Dis_iter = []
#     Iter = 0
    while it < 800:
        success = 0 
        for i in range(n):
            P_acc_iter.append(P_acc)
            Res_loss.append(E)
            V_iter.append(Vol)
            Str_iter.append(Str)
            Dis_iter.append(Dis)
            prev_state = copy.deepcopy(state)
        #             print('starting state of cycle',prev_state)
            Total=len(Unit)
#             Int = 175
#             Tun = 450
#             if i == 0:
#                 print('New Objective:',E)
#                 if T == Int:
#                     print('Intermediate Zone')
#                 elif T< Tun:
#                     print('Tuning Zone')
            Int = T_0/2
            Tun = 1
            if i == 0:
                print('New Objective:',E)
            if T<=T_0 and T > Int:
                RANGE = int(Total/4)
            elif T>Tun and T<= Int:
                RANGE = int(Total/40)
            else:
                RANGE = int(Total/400)
            new_state = copy.deepcopy(state)
            for I in range (RANGE):
                Size = np.random.randint(1,3)
                if Size == 2:
                    rand_cell_select = np.random.randint(0,len(Sup)) #randomly select unit cell
                else:
                    rand_cell_select = np.random.randint(len(Sup),len(Unit)-len(Sup)) #randomly select unit cell
                new_cell = {'CellType':[],'Volume':[]} #initialize new cell
                new_cell = cell(new_cell,Size) #obtain new random cell
                new_state['CellType'][rand_cell_select]=new_cell['CellType'][0]
                new_state['Volume'][rand_cell_select]=new_cell['Volume'][0]
            Vol2,ver2 = constraint(new_state,V_max)
#                 print('constraint:',ver2)
            if ver2 == True:
                test,E_curr,Vol_curr, P_acc,Dis_curr, Str_curr = metopolis(E,Vol,Vol2,new_state,prev_state,T,init,Dis,Str,DisMax,StrMax)
                E = E_curr
                Vol = Vol_curr
                Dis = Dis_curr
                Str = Str_curr
#                     print('Objective',E)
            else:
                test = False
            it+=1 
            if test == True:
                state = new_state
                success += 1
            else:
                state = prev_state
#             print('Successes',success)

            if success > limit:
                print('~~~~~~~~New Temp~~~~~~~~~~')
                break
        if success == 0:
#             print('Success = 0')
            break
        T = T*alpha
    print('Final Objective',E,'Final Volume',np.round(Vol,3), 'Final Displacement', np.round(Dis,3), 'Final Stress', Str)
    print('Final State',state)
    return state,Res_loss,V_iter,P_acc_iter,Str_iter,Dis_iter


In [72]:
##### 
start_time = time.time()
T = 50
n = 100

limit = n/2
alpha = 0.5
np.set_printoptions(threshold=sys.maxsize)
CellCount = []
SOLU = []
start = 0
end = 5
for RUN in range(start,end):
    start_time = time.time()
    titleS = f"{os.getcwd()}\\AsymSupport.txt"
    Sup = np.loadtxt(titleS)
    titleS2 = f"{os.getcwd()}\\AsymSingle.txt"
    Sing = np.loadtxt(titleS2)
    Unit=np.concatenate((Sup,Sing))

    

    %run ./AA_Adapter_MAPDL.ipynb
    mapdl.exit()
    path = os.getcwd()
    mapdl = launch_mapdl(nproc = 8, run_location=path,override = True, license_type = "ansys")
    mapdl.clear()           
    
    DisMax = 40
    StrMax = 0.66
    V_max = 3808
    init = 0
    Vol,state = initialize(Unit,Sup,init)
    print('state',state)

    E,init,Dis,Str= evaluate(state,init,DisMax,StrMax)
    init = 1
    print('Initial Objective',E,'Volume',np.round(Vol,3),'Max Displacement',np.round(Dis,3),'Max Stress',Str)
    CELL = np.array(state['CellType'])
    sc = np.count_nonzero(CELL == 'SC')
    tr = np.count_nonzero(CELL == 'TR')
    fc = np.count_nonzero(CELL == 'FC')
    sc1 = np.count_nonzero(CELL == 'sc')
    bv1 = np.count_nonzero(CELL == 'bc')
    sol = np.count_nonzero(CELL == 'sol')
    Cell_count = [sc,tr,fc,sc1,bv1,sol]
    CellCount.append(Cell_count)
    print('Simple Cubic:', sc,'Transition Cubic:', tr,'Face-Centered Vertile Cubic:',fc)
    print('SC1:', sc1,'BV1 Cubic:', bv1,'Solid Cubic:',sol)
    it = 1
    solutions,R_loss,V_iter,P_acc_iter,Str_iter,Dis_iter= ShapeAnneal(it,state,T,limit,n,alpha,Sup,Unit,E,Vol,init,V_max,Dis,Str,DisMax,StrMax)
    mapdl.exit()
    CELL = np.array(solutions['CellType'])
    SOLU.append(solutions)
    sc = np.count_nonzero(CELL == 'SC')
    tr = np.count_nonzero(CELL == 'TR')
    fc = np.count_nonzero(CELL == 'FC')
    sc1 = np.count_nonzero(CELL == 'sc')
    bv1 = np.count_nonzero(CELL == 'bc')
    sol = np.count_nonzero(CELL == 'sol')
    Cell_count = [sc,tr,fc,sc1,bv1,sol]
    CellCount.append(Cell_count)
    print('Simple Cubic:', sc,'Transition Cubic:', tr,'Face-Centered Vertile Cubic:',fc)
    print('SC1:', sc1,'BV1 Cubic:', bv1,'Solid Cubic:',sol)
    print("--- %s seconds ---" % (time.time() - start_time))    
    Iter_data = np.concatenate((np.array(R_loss).reshape(-1,1),np.round(np.array(V_iter).reshape(-1,1),3),np.round(np.array(Str_iter).reshape(-1,1),3),np.round(np.array(Dis_iter).reshape(-1,1),3)),axis = 1)
    with open('ASymm50_4_%i' %RUN + '.txt', 'w') as f:
        content = str(Iter_data)
        f.write(content)
        f.close()
CC = np.array(CellCount)
with open('CellCount_ASymm50_4_%i'%start+'-%i'%end+'.txt', 'w') as f:
    content = str(CC)
    f.write(content)
    f.close()
with open('SOLU_ASymm50_4_%i'%start+'-%i'%end+'.txt', 'w') as f:
    content = str(SOLU)
    f.write(content)
    f.close()

state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'S

Structural boundary conditions attached--- 1.0282509326934814 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2930.81610676]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3083.345577]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3271.5026746]
New Objective: [-3384.45862221]
New Objective: [-3488.23489385]
New Objective: [-3549.91570115]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3627.43021689]
New Objective: [-3702.51586252]
Final Objective [-3724.01850378] Final Volume [3806.987] Final Displacement 16.631 Final Stress 0.09675386844880164
Final State {'CellType': ['FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'SC', 'TR', 'TR', 'FC', 'FC', 'FC', 'FC', 'FC', 'TR', 'FC', 'TR', 'FC', 'TR', 'FC', 'TR', 'FC', 'FC', 'FC', 'SC', 'SC', 'SC', 'SC', 'TR', 'T

Simple Cubic: 145 Transition Cubic: 181 Face-Centered Vertile Cubic: 305
SC1: 630 BV1 Cubic: 1255 Solid Cubic: 678
--- 42515.98212695122 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC

Structural boundary conditions attached--- 1.0466299057006836 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3023.85635442]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3081.1428027]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3155.80389395]
New Objective: [-3263.78984403]
New Objective: [-3391.61571927]
New Objective: [-3457.50998483]
New Objective: [-3574.94793152]
New Objective: [-3643.73249472]
Final Objective [-3696.75293342] Final Volume [3792.473] Final Displacement 15.61 Final Stress 0.08498662451003182
Final State {'CellType': ['FC', 'FC', 'FC', 'FC', 'SC', 'TR', 'TR', 'FC', 'TR', 'SC', 'FC', 'SC', 'TR', 'FC', 'FC', 'TR', 'TR', 'FC', 'TR', 'TR', 'TR', 'SC', 'SC', 'FC', 'SC', 'FC', 'SC', 'FC', 'TR', 'TR', 'FC', 'FC', 'FC', 'FC'

Simple Cubic: 158 Transition Cubic: 186 Face-Centered Vertile Cubic: 287
SC1: 645 BV1 Cubic: 1218 Solid Cubic: 700
--- 43147.9035949707 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC'

...

Simple Cubic: 154 Transition Cubic: 167 Face-Centered Vertile Cubic: 310
SC1: 638 BV1 Cubic: 1235 Solid Cubic: 690
--- 41446.665804862976 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'S

Structural boundary conditions attached--- 1.0466301441192627 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3078.36236082]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3093.61845267]
New Objective: [-3239.99913079]
New Objective: [-3396.93207131]
New Objective: [-3540.66380244]
New Objective: [-3591.68317183]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3683.99776264]
New Objective: [-3741.94254627]
Final Objective [-3771.86523889] Final Volume [3802.955] Final Displacement 16.157 Final Stress 0.08856342174400783
Final State {'CellType': ['FC', 'FC', 'TR', 'TR', 'FC', 'FC', 'FC', 'FC', 'TR', 'FC', 'SC', 'FC', 'SC', 'FC', 'FC', 'TR', 'FC', 'TR', 'FC', 'FC', 'FC', 'SC', 'SC', 'TR', 'FC', 'TR', 'FC', 'SC', 'TR', 'FC', 'FC', 'TR', 'FC', 'S

Simple Cubic: 148 Transition Cubic: 173 Face-Centered Vertile Cubic: 310
SC1: 668 BV1 Cubic: 1245 Solid Cubic: 650
--- 42440.69471883774 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC

Structural boundary conditions attached--- 1.031008243560791 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3067.39456571]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3207.94341058]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3303.15558877]
New Objective: [-3313.50450875]
New Objective: [-3423.07518878]
New Objective: [-3483.29791342]
New Objective: [-3566.3290119]
New Objective: [-3666.69181676]
Final Objective [-3761.45378462] Final Volume [3800.825] Final Displacement 15.531 Final Stress 0.08548948466559213
Final State {'CellType': ['FC', 'FC', 'FC', 'SC', 'TR', 'FC', 'TR', 'FC', 'FC', 'TR', 'TR', 'SC', 'FC', 'FC', 'FC', 'SC', 'FC', 'TR', 'FC', 'SC', 'TR', 'FC', 'FC', 'TR', 'TR', 'TR', 'SC', 'FC', 'FC', 'FC', 'TR', 'TR', 'TR', 'SC'

Simple Cubic: 159 Transition Cubic: 170 Face-Centered Vertile Cubic: 302
SC1: 676 BV1 Cubic: 1204 Solid Cubic: 683
--- 43149.151465177536 seconds ---


In [75]:
##### 
start_time = time.time()
T = 50
n = 100

limit = n/2
alpha = 0.5
np.set_printoptions(threshold=sys.maxsize)
CellCount = []
SOLU = []
start = 10
end = 15
for RUN in range(start,end):
    start_time = time.time()
    titleS = f"{os.getcwd()}\\AsymSupport.txt"
    Sup = np.loadtxt(titleS)
    titleS2 = f"{os.getcwd()}\\AsymSingle.txt"
    Sing = np.loadtxt(titleS2)
    Unit=np.concatenate((Sup,Sing))

    

    %run ./AA_Adapter_MAPDL.ipynb
    mapdl.exit()
    path = os.getcwd()
    mapdl = launch_mapdl(nproc = 8, run_location=path,override = True, license_type = "ansys")
    mapdl.clear()           
    
    DisMax = 40
    StrMax = 0.66
    V_max = 3808
    init = 0
    Vol,state = initialize(Unit,Sup,init)
    print('state',state)

    E,init,Dis,Str= evaluate(state,init,DisMax,StrMax)
    init = 1
    print('Initial Objective',E,'Volume',np.round(Vol,3),'Max Displacement',np.round(Dis,3),'Max Stress',Str)
    CELL = np.array(state['CellType'])
    sc = np.count_nonzero(CELL == 'SC')
    tr = np.count_nonzero(CELL == 'TR')
    fc = np.count_nonzero(CELL == 'FC')
    sc1 = np.count_nonzero(CELL == 'sc')
    bv1 = np.count_nonzero(CELL == 'bc')
    sol = np.count_nonzero(CELL == 'sol')
    Cell_count = [sc,tr,fc,sc1,bv1,sol]
    CellCount.append(Cell_count)
    print('Simple Cubic:', sc,'Transition Cubic:', tr,'Face-Centered Vertile Cubic:',fc)
    print('SC1:', sc1,'BV1 Cubic:', bv1,'Solid Cubic:',sol)
    it = 1
    solutions,R_loss,V_iter,P_acc_iter,Str_iter,Dis_iter= ShapeAnneal(it,state,T,limit,n,alpha,Sup,Unit,E,Vol,init,V_max,Dis,Str,DisMax,StrMax)
    mapdl.exit()
    CELL = np.array(solutions['CellType'])
    SOLU.append(solutions)
    sc = np.count_nonzero(CELL == 'SC')
    tr = np.count_nonzero(CELL == 'TR')
    fc = np.count_nonzero(CELL == 'FC')
    sc1 = np.count_nonzero(CELL == 'sc')
    bv1 = np.count_nonzero(CELL == 'bc')
    sol = np.count_nonzero(CELL == 'sol')
    Cell_count = [sc,tr,fc,sc1,bv1,sol]
    CellCount.append(Cell_count)
    print('Simple Cubic:', sc,'Transition Cubic:', tr,'Face-Centered Vertile Cubic:',fc)
    print('SC1:', sc1,'BV1 Cubic:', bv1,'Solid Cubic:',sol)
    print("--- %s seconds ---" % (time.time() - start_time))    
    Iter_data = np.concatenate((np.array(R_loss).reshape(-1,1),np.round(np.array(V_iter).reshape(-1,1),3),np.round(np.array(Str_iter).reshape(-1,1),3),np.round(np.array(Dis_iter).reshape(-1,1),3)),axis = 1)
    with open('ASymm50_4_%i' %RUN + '.txt', 'w') as f:
        content = str(Iter_data)
        f.write(content)
        f.close()
CC = np.array(CellCount)
with open('CellCount_ASymm50_4_%i'%start+'-%i'%end+'.txt', 'w') as f:
    content = str(CC)
    f.write(content)
    f.close()
with open('SOLU_ASymm50_4_%i'%start+'-%i'%end+'.txt', 'w') as f:
    content = str(SOLU)
    f.write(content)
    f.close()

state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'S

Structural boundary conditions attached--- 1.0313973426818848 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3123.04018814]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3063.91122902]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3225.84210209]
New Objective: [-3344.0790662]
New Objective: [-3441.78013013]
New Objective: [-3498.05161636]
New Objective: [-3597.02827429]
New Objective: [-3695.89380341]
Final Objective [-3719.12466842] Final Volume [3806.618] Final Displacement 16.41 Final Stress 0.08419782873606965
Final State {'CellType': ['FC', 'FC', 'SC', 'FC', 'FC', 'FC', 'FC', 'FC', 'SC', 'SC', 'SC', 'TR', 'FC', 'FC', 'TR', 'FC', 'FC', 'FC', 'FC', 'FC', 'TR', 'TR', 'SC', 'TR', 'TR', 'FC', 'FC', 'TR', 'FC', 'TR', 'FC', 'FC', 'FC', 'FC'

Simple Cubic: 152 Transition Cubic: 199 Face-Centered Vertile Cubic: 280
SC1: 626 BV1 Cubic: 1201 Solid Cubic: 736
--- 39522.084864377975 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'S

Structural boundary conditions attached--- 1.0310084819793701 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2997.46863958]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2969.72271436]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3143.91641336]
New Objective: [-3296.58060016]
New Objective: [-3370.96055613]
New Objective: [-3516.70725437]
New Objective: [-3618.53078578]
New Objective: [-3680.58775773]
Final Objective [-3770.85332434] Final Volume [3802.574] Final Displacement 16.473 Final Stress 0.08175055740901616
Final State {'CellType': ['FC', 'FC', 'FC', 'FC', 'FC', 'SC', 'TR', 'TR', 'FC', 'TR', 'FC', 'FC', 'FC', 'FC', 'FC', 'SC', 'FC', 'FC', 'TR', 'FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'TR', 'FC', 'FC', 'FC', 'TR', 'FC', 'TR', 'TR', 'F

Simple Cubic: 146 Transition Cubic: 193 Face-Centered Vertile Cubic: 292
SC1: 664 BV1 Cubic: 1226 Solid Cubic: 673
--- 41166.394906044006 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'S

...

Simple Cubic: 159 Transition Cubic: 191 Face-Centered Vertile Cubic: 281
SC1: 636 BV1 Cubic: 1273 Solid Cubic: 654
--- 42816.720645189285 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'S

Structural boundary conditions attached--- 0.9997661113739014 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2957.67590281]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3054.97908305]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3171.50068075]
New Objective: [-3273.7874186]
New Objective: [-3415.99563197]
New Objective: [-3530.23738638]
New Objective: [-3587.82523067]
New Objective: [-3674.84730831]
Final Objective [-3731.46854188] Final Volume [3806.622] Final Displacement 15.436 Final Stress 0.08588477940416349
Final State {'CellType': ['FC', 'FC', 'TR', 'FC', 'TR', 'SC', 'FC', 'FC', 'FC', 'FC', 'SC', 'SC', 'SC', 'TR', 'FC', 'FC', 'FC', 'TR', 'FC', 'TR', 'FC', 'SC', 'SC', 'SC', 'FC', 'SC', 'FC', 'FC', 'FC', 'TR', 'FC', 'TR', 'SC', 'FC

Simple Cubic: 151 Transition Cubic: 180 Face-Centered Vertile Cubic: 300
SC1: 671 BV1 Cubic: 1212 Solid Cubic: 680
--- 42285.788264751434 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'S

Structural boundary conditions attached--- 1.0310075283050537 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3089.24349159]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3148.05044113]
New Objective: [-3317.54674925]
New Objective: [-3396.95220392]
New Objective: [-3425.81061195]
New Objective: [-3497.96970891]
New Objective: [-3572.22950828]
New Objective: [-3625.49773255]
Final Objective [-3661.82009506] Final Volume [3804.472] Final Displacement 15.47 Final Stress 0.09055286067942864
Final State {'CellType': ['FC', 'FC', 'FC', 'TR', 'FC', 'FC', 'TR', 'FC', 'FC', 'FC', 'SC', 'TR', 'FC', 'TR', 'FC', 'FC', 'FC', 'TR', 'FC', 'FC', 'TR', 'TR', 'FC', 'TR', 'FC', 'FC', 'TR', 'FC', 'FC', 'FC', 'TR', 'FC', 'SC', 'FC', 'TR', 'FC', 'FC', 'FC', 

Simple Cubic: 135 Transition Cubic: 215 Face-Centered Vertile Cubic: 281
SC1: 674 BV1 Cubic: 1230 Solid Cubic: 659
--- 42859.500509262085 seconds ---


In [ ]:
##### 
start_time = time.time()
T = 50
n = 100

limit = n/2
alpha = 0.5
np.set_printoptions(threshold=sys.maxsize)
CellCount = []
SOLU = []
start = 20
end = 25
for RUN in range(start,end):
    start_time = time.time()
    titleS = f"{os.getcwd()}\\AsymSupport.txt"
    Sup = np.loadtxt(titleS)
    titleS2 = f"{os.getcwd()}\\AsymSingle.txt"
    Sing = np.loadtxt(titleS2)
    Unit=np.concatenate((Sup,Sing))

    

    %run ./AA_Adapter_MAPDL.ipynb
    mapdl.exit()
    path = os.getcwd()
    mapdl = launch_mapdl(nproc = 8, run_location=path,override = True, license_type = "ansys")
    mapdl.clear()           
    
    DisMax = 40
    StrMax = 0.66
    V_max = 3808
    init = 0
    Vol,state = initialize(Unit,Sup,init)
    print('state',state)

    E,init,Dis,Str= evaluate(state,init,DisMax,StrMax)
    init = 1
    print('Initial Objective',E,'Volume',np.round(Vol,3),'Max Displacement',np.round(Dis,3),'Max Stress',Str)
    CELL = np.array(state['CellType'])
    sc = np.count_nonzero(CELL == 'SC')
    tr = np.count_nonzero(CELL == 'TR')
    fc = np.count_nonzero(CELL == 'FC')
    sc1 = np.count_nonzero(CELL == 'sc')
    bv1 = np.count_nonzero(CELL == 'bc')
    sol = np.count_nonzero(CELL == 'sol')
    Cell_count = [sc,tr,fc,sc1,bv1,sol]
    CellCount.append(Cell_count)
    print('Simple Cubic:', sc,'Transition Cubic:', tr,'Face-Centered Vertile Cubic:',fc)
    print('SC1:', sc1,'BV1 Cubic:', bv1,'Solid Cubic:',sol)
    it = 1
    solutions,R_loss,V_iter,P_acc_iter,Str_iter,Dis_iter= ShapeAnneal(it,state,T,limit,n,alpha,Sup,Unit,E,Vol,init,V_max,Dis,Str,DisMax,StrMax)
    mapdl.exit()
    CELL = np.array(solutions['CellType'])
    SOLU.append(solutions)
    sc = np.count_nonzero(CELL == 'SC')
    tr = np.count_nonzero(CELL == 'TR')
    fc = np.count_nonzero(CELL == 'FC')
    sc1 = np.count_nonzero(CELL == 'sc')
    bv1 = np.count_nonzero(CELL == 'bc')
    sol = np.count_nonzero(CELL == 'sol')
    Cell_count = [sc,tr,fc,sc1,bv1,sol]
    CellCount.append(Cell_count)
    print('Simple Cubic:', sc,'Transition Cubic:', tr,'Face-Centered Vertile Cubic:',fc)
    print('SC1:', sc1,'BV1 Cubic:', bv1,'Solid Cubic:',sol)
    print("--- %s seconds ---" % (time.time() - start_time))    
    Iter_data = np.concatenate((np.array(R_loss).reshape(-1,1),np.round(np.array(V_iter).reshape(-1,1),3),np.round(np.array(Str_iter).reshape(-1,1),3),np.round(np.array(Dis_iter).reshape(-1,1),3)),axis = 1)
    with open('ASymm50_4_%i' %RUN + '.txt', 'w') as f:
        content = str(Iter_data)
        f.write(content)
        f.close()
CC = np.array(CellCount)
with open('CellCount_ASymm50_4_%i'%start+'-%i'%end+'.txt', 'w') as f:
    content = str(CC)
    f.write(content)
    f.close()
with open('SOLU_ASymm50_4_%i'%start+'-%i'%end+'.txt', 'w') as f:
    content = str(SOLU)
    f.write(content)
    f.close()

state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'S

Structural boundary conditions attached--- 1.0402190685272217 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3005.28804354]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3168.4866761]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3191.46067511]
New Objective: [-3420.58029819]
New Objective: [-3503.30345974]
New Objective: [-3558.20461773]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3664.77846297]
New Objective: [-3751.1887381]
Final Objective [-3799.03620916] Final Volume [3807.045] Final Displacement 16.069 Final Stress 0.0937053978544631
Final State {'CellType': ['TR', 'FC', 'SC', 'FC', 'FC', 'FC', 'SC', 'FC', 'FC', 'FC', 'SC', 'SC', 'SC', 'FC', 'FC', 'SC', 'SC', 'FC', 'TR', 'FC', 'TR', 'TR', 'FC', 'SC', 'FC', 'SC', 'FC', 'FC', 'FC', 'S

Simple Cubic: 151 Transition Cubic: 182 Face-Centered Vertile Cubic: 298
SC1: 628 BV1 Cubic: 1229 Solid Cubic: 706
--- 42239.57321476936 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC

Structural boundary conditions attached--- 1.031008005142212 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3109.50087818]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3215.33092111]
New Objective: [-3266.69935759]
New Objective: [-3339.42594665]
New Objective: [-3408.6658268]
New Objective: [-3545.98311394]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3612.08298039]
New Objective: [-3737.92860486]
Final Objective [-3777.22712352] Final Volume [3807.034] Final Displacement 16.185 Final Stress 0.08777279833724436
Final State {'CellType': ['FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'TR', 'TR', 'SC', 'SC', 'FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'TR', 'TR', 'FC', 'TR', 'TR', 'FC'

Simple Cubic: 151 Transition Cubic: 182 Face-Centered Vertile Cubic: 298
SC1: 626 BV1 Cubic: 1230 Solid Cubic: 707
--- 43031.24802899361 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC

Structural boundary conditions attached--- 0.9685230255126953 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3131.39743135]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3086.97062957]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3250.88488348]
New Objective: [-3359.42656881]
New Objective: [-3465.14666984]
New Objective: [-3515.10894901]
New Objective: [-3601.24266858]
New Objective: [-3670.88898772]
Final Objective [-3734.93761569] Final Volume [3792.993] Final Displacement 16.675 Final Stress 0.08294651557383952
Final State {'CellType': ['FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'TR', 'SC', 'FC', 'TR', 'FC', 'FC', 'FC', 'TR', 'SC', 'FC', 'TR', 'FC', 'FC', 'TR', 'FC', 'SC', 'FC', 'SC', 'FC', 'FC', 'TR', 'SC', 'TR', 'TR', 'TR', 'FC', 'TR', 'S

Simple Cubic: 160 Transition Cubic: 172 Face-Centered Vertile Cubic: 299
SC1: 640 BV1 Cubic: 1232 Solid Cubic: 691
--- 43275.259536743164 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'S

Structural boundary conditions attached--- 1.0466289520263672 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3102.37167939]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3080.28867209]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3209.78642244]
New Objective: [-3273.26761998]
New Objective: [-3422.3239477]
New Objective: [-3508.46207532]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3625.23427668]
New Objective: [-3683.75695897]
Final Objective [-3727.88743578] Final Volume [3806.456] Final Displacement 15.945 Final Stress 0.08131985523886257
Final State {'CellType': ['FC', 'FC', 'FC', 'TR', 'FC', 'TR', 'FC', 'FC', 'TR', 'TR', 'SC', 'TR', 'TR', 'TR', 'FC', 'SC', 'SC', 'FC', 'TR', 'SC', 'TR', 'TR', 'FC', 'SC', 'FC', 'TR', 'FC', 'FC', 'TR', 

Simple Cubic: 141 Transition Cubic: 206 Face-Centered Vertile Cubic: 284
SC1: 649 BV1 Cubic: 1226 Solid Cubic: 688
--- 42687.263959646225 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'S

Structural boundary conditions attached--- 1.0310075283050537 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3084.28598993]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3194.76927818]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3323.30432068]
New Objective: [-3375.17399038]
New Objective: [-3478.49207009]
New Objective: [-3539.67675978]
New Objective: [-3586.3498771]
New Objective: [-3642.80655762]


In [11]:
#### 
start_time = time.time()
T = 50
n = 100

limit = n/2
alpha = 0.5
np.set_printoptions(threshold=sys.maxsize)
CellCount = []
SOLU = []
start = 24
end = 26
for RUN in range(start,end):
    start_time = time.time()
    titleS = f"{os.getcwd()}\\AsymSupport.txt"
    Sup = np.loadtxt(titleS)
    titleS2 = f"{os.getcwd()}\\AsymSingle.txt"
    Sing = np.loadtxt(titleS2)
    Unit=np.concatenate((Sup,Sing))

    

    %run ./AA_Adapter_MAPDL.ipynb
#     mapdl.exit()
    path = os.getcwd()
    mapdl = launch_mapdl(nproc = 8, run_location=path,override = True, license_type = "ansys")
    mapdl.clear()           
    
    DisMax = 40
    StrMax = 0.66
    V_max = 3808
    init = 0
    Vol,state = initialize(Unit,Sup,init)
    print('state',state)

    E,init,Dis,Str= evaluate(state,init,DisMax,StrMax)
    init = 1
    print('Initial Objective',E,'Volume',np.round(Vol,3),'Max Displacement',np.round(Dis,3),'Max Stress',Str)
    CELL = np.array(state['CellType'])
    sc = np.count_nonzero(CELL == 'SC')
    tr = np.count_nonzero(CELL == 'TR')
    fc = np.count_nonzero(CELL == 'FC')
    sc1 = np.count_nonzero(CELL == 'sc')
    bv1 = np.count_nonzero(CELL == 'bc')
    sol = np.count_nonzero(CELL == 'sol')
    Cell_count = [sc,tr,fc,sc1,bv1,sol]
    CellCount.append(Cell_count)
    print('Simple Cubic:', sc,'Transition Cubic:', tr,'Face-Centered Vertile Cubic:',fc)
    print('SC1:', sc1,'BV1 Cubic:', bv1,'Solid Cubic:',sol)
    it = 1
    solutions,R_loss,V_iter,P_acc_iter,Str_iter,Dis_iter= ShapeAnneal(it,state,T,limit,n,alpha,Sup,Unit,E,Vol,init,V_max,Dis,Str,DisMax,StrMax)
    mapdl.exit()
    CELL = np.array(solutions['CellType'])
    SOLU.append(solutions)
    sc = np.count_nonzero(CELL == 'SC')
    tr = np.count_nonzero(CELL == 'TR')
    fc = np.count_nonzero(CELL == 'FC')
    sc1 = np.count_nonzero(CELL == 'sc')
    bv1 = np.count_nonzero(CELL == 'bc')
    sol = np.count_nonzero(CELL == 'sol')
    Cell_count = [sc,tr,fc,sc1,bv1,sol]
    CellCount.append(Cell_count)
    print('Simple Cubic:', sc,'Transition Cubic:', tr,'Face-Centered Vertile Cubic:',fc)
    print('SC1:', sc1,'BV1 Cubic:', bv1,'Solid Cubic:',sol)
    print("--- %s seconds ---" % (time.time() - start_time))    
    Iter_data = np.concatenate((np.array(R_loss).reshape(-1,1),np.round(np.array(V_iter).reshape(-1,1),3),np.round(np.array(Str_iter).reshape(-1,1),3),np.round(np.array(Dis_iter).reshape(-1,1),3)),axis = 1)
    with open('ASymm50_4_%i' %RUN + '.txt', 'w') as f:
        content = str(Iter_data)
        f.write(content)
        f.close()
CC = np.array(CellCount)
with open('CellCount_ASymm50_4_%i'%start+'-%i'%end+'.txt', 'w') as f:
    content = str(CC)
    f.write(content)
    f.close()
with open('SOLU_ASymm50_4_%i'%start+'-%i'%end+'.txt', 'w') as f:
    content = str(SOLU)
    f.write(content)
    f.close()

state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'S

Structural boundary conditions attached--- 0.9685227870941162 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3118.49844995]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3166.25915238]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3242.78820205]
New Objective: [-3344.32519669]
New Objective: [-3484.74223426]
New Objective: [-3566.57799524]
New Objective: [-3662.11504585]
New Objective: [-3718.59004757]
Final Objective [-3774.79765821] Final Volume [3807.124] Final Displacement 16.056 Final Stress 0.08524252006509048
Final State {'CellType': ['FC', 'FC', 'TR', 'FC', 'TR', 'TR', 'FC', 'FC', 'TR', 'FC', 'SC', 'TR', 'FC', 'TR', 'TR', 'FC', 'FC', 'TR', 'SC', 'FC', 'SC', 'FC', 'SC', 'SC', 'TR', 'FC', 'FC', 'TR', 'TR', 'FC', 'SC', 'SC', 'FC', 'S

Simple Cubic: 158 Transition Cubic: 170 Face-Centered Vertile Cubic: 303
SC1: 676 BV1 Cubic: 1194 Solid Cubic: 693
--- 39457.84341263771 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC

Structural boundary conditions attached--- 1.0310084819793701 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3000.80967568]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3039.45234785]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3147.88259642]
New Objective: [-3345.66512207]
New Objective: [-3437.11620987]
New Objective: [-3586.84131028]
New Objective: [-3659.18072148]
New Objective: [-3727.08357778]
Final Objective [-3765.61263278] Final Volume [3804.476] Final Displacement 15.956 Final Stress 0.09434783275803629
Final State {'CellType': ['FC', 'TR', 'FC', 'TR', 'FC', 'SC', 'FC', 'TR', 'FC', 'FC', 'FC', 'SC', 'FC', 'FC', 'FC', 'TR', 'SC', 'FC', 'FC', 'SC', 'TR', 'SC', 'FC', 'SC', 'SC', 'TR', 'FC', 'SC', 'TR', 'FC', 'TR', 'FC', 'TR', 'T

Simple Cubic: 150 Transition Cubic: 173 Face-Centered Vertile Cubic: 308
SC1: 658 BV1 Cubic: 1238 Solid Cubic: 667
--- 41224.524131536484 seconds ---
